In [2]:
# !pip install weaviate-client
# !pip install llama_index
# !pip install openai
# !pip install --upgrade pydantic==1.10.0 typing-extensions==4.5.0
# !pip install fastcore

In [2]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index.schema import TextNode, NodeRelationship, RelatedNodeInfo
from llama_index import VectorStoreIndex, StorageContext
from llama_index import get_response_synthesizer
from llama_index.indices.postprocessor import SimilarityPostprocessor, PrevNextNodePostprocessor
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.embeddings import HuggingFaceEmbedding
import weaviate, os
from weaviate import EmbeddedOptions
from openai import OpenAI
import pandas as pd
import numpy as np
from llama_index import ServiceContext
from llama_index.llms import OpenAI as OpenAILLama
from llama_index.embeddings import OpenAIEmbedding
from configparser import ConfigParser
import openai
import os

pd.set_option('display.max_colwidth',-1)

/tmp/ipykernel_341/1637876314.py:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


## OpenAI account

In [3]:
config=ConfigParser()
config.read('conf/conf.ini')
os.environ["OPENAI_API_KEY"] = config['openai']['apikey']

## Setup weaviate client

In [4]:
# !rm -rf ~/.local/share/weaviate

In [5]:
client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    # additional_headers={
    #     "X-OpenAI-Api-BaseURL": os.environ['OPENAI_API_BASE'],
    #     "X-OpenAI-Api-Key": openai.api_key,  # Replace this with your actual key
    # }
)
print(f"Client created? {client.is_ready()}")

{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2023-12-13T18:23:27Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2023-12-13T18:23:27Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2023-12-13T18:23:27Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2023-12-13T18:23:27Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2023-12-13T18:23:27Z"}


Started /root/.cache/weaviate-embedded: process ID 365
Client created? True


## Insert data weaviate

In [6]:
embed_model=HuggingFaceEmbedding(model_name='intfloat/multilingual-e5-base')

In [7]:
df=pd.read_excel('data/riigikogu/ems_subset.xlsx')
# embs=np.load('data/riigikogu/emsb.npy', allow_pickle=True)

/usr/lib/python3/dist-packages/openpyxl/compat/numbers.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.float,


In [8]:
df.shape, #embs.shape

((1000, 1),)

In [9]:
data=[{'text':row['text_splitted'],'foo':i} for i, row in df.iterrows()]

In [10]:
nodes = []
node_prev=None

for idx, sample in enumerate(data):
    node = TextNode(
        #id_=idx,
        text=sample['text'],
    )
    node.embedding=embed_model.get_text_embedding(sample['text'])
    # if idx< len(data)-1:
    # node.relationships[NodeRelationship.NEXT]=idx+1
    if idx>0:
        node.relationships[NodeRelationship.PREVIOUS]=RelatedNodeInfo(node_id=node_prev.node_id)
        node_prev.relationships[NodeRelationship.NEXT]=RelatedNodeInfo(node_id=node.node_id)
    nodes.append(node)
    node_prev=node.copy()

In [11]:
# model.encode('tere')

In [12]:
node.relationships

{<NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='1bf84cb7-b07f-4a32-acac-7f5296182c15', node_type=None, metadata={}, hash=None)}

In [13]:
NodeRelationship.PREVIOUS

<NodeRelationship.PREVIOUS: '2'>

In [14]:
nodes[2].relationships

{<NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='e846221d-2929-4b18-888e-95e516672353', node_type=None, metadata={}, hash=None),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='028e2930-a990-45cc-90f1-1224fed16e72', node_type=None, metadata={}, hash=None)}

In [15]:
nodes[3].relationships

{<NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='634b0434-0f29-4297-b629-3bf84634e36e', node_type=None, metadata={}, hash=None),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='f323aefb-a8e8-4fcf-986e-3b4c2228f82b', node_type=None, metadata={}, hash=None)}

In [16]:
RelatedNodeInfo(node_id=node_prev.node_id)

RelatedNodeInfo(node_id='85a9518c-d8ea-4dc1-9113-89f2bfb5d87e', node_type=None, metadata={}, hash=None)

## Create vector store

In [17]:
llm = OpenAILLama(model="gpt-3.5-turbo", temperature=0.1)
# embed_model=OpenAIEmbedding()


service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

In [20]:
# embed_model.get_text_embedding('tere')

In [21]:
#if already run from the start make sure to delete exsiting database
# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="Riigikogu", text_key="text")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context, service_context=service_context)

## If needed setup syntheziser, retriever, post-processors

In [22]:
# Set up the synthesizer
# my_synthesizer = get_response_synthesizer(response_mode="tree_summarize")
# to get only response nodes
my_synthesizer = get_response_synthesizer(response_mode='no_text')

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=20,
)

postprocessor = PrevNextNodePostprocessor(
    docstore=index.docstore,
    num_nodes=1,  # number of nodes to fetch when looking forawrds or backwards
    mode="both",  # can be either 'next', 'previous', or 'both'
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=my_synthesizer,
    node_postprocessors=[
        #postprocessor,
        SimilarityPostprocessor(similarity_cutoff=0.8),
    ]
)

#### example query

In [23]:
query_str = "milliseid ettepanekuid arutatakse, et riigieelarve tasakaalu saada?"
response=query_engine.query(query_str)

In [24]:
response.response

''

In [25]:
response.source_nodes[0].text

'kindlasti hoida nii et vaatame maksubaasi ja vaatame kui palju raha juurde saab aga süsteem ise peab olema piisavalt lihtne selleks et ta oleks efektiivne'

In [26]:
len(response.source_nodes)

20

In [27]:
def nodes2df(nodes:list):
    """format list of nodes to pandas dataframe"""
    nodes_fmt=[(n.text, n.id_, n.score) for n in nodes]
    return pd.DataFrame(nodes_fmt, columns=['text','id', 'score'])

In [28]:
# nodes2df(response.source_nodes)

## Setup function calls

In [29]:
import ast
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [30]:
client_oai = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
client_oai

In [31]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [32]:
funcs_ok = {'ask_query_count', 'python'}

#TODOD make queryengine as an argument
def ask_query_count(query_str:str):
    """This function is useful to find count of texts in the database that match query_str. This function gives access to databased.
    Query is made into vector database, returns reponse nodes and counts them
    Query_str should contain words or phrases to search for. It doesn't have to contain AND, OR etc. Could be freeform text"""
    print(f'query string for ask_query_count is {query_str}')
    response=query_engine.query(query_str)
    df_resp=nodes2df(response.source_nodes)
    return str(len(response.source_nodes)), df_resp

In [33]:
def askgpt_func_call(user, system="you are helpful assistant and answer questions about the database that you can access by calling function ask_query_count", model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    c=client_oai.chat.completions.create(model=model, messages=msgs, **kwargs)
    
    if c.choices[0].message.function_call is None:
        return c.choices[0].message.content
    func_call_out, df_resp=call_func(c)
    msgs.append({"role": "function", "name": "python", "content": func_call_out})
    c2=client_oai.chat.completions.create(model=model, messages=msgs)
    return c2.choices[0].message.content, func_call_out, df_resp

In [34]:
resp, func_call_out, df_resp=askgpt_func_call('How many texts mention "sõda" or "kaitsevägi"?',
                functions=[schema(ask_query_count)])
resp

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
query string for ask_query_count is sõda OR kaitsevägi


'There are 20 texts that mention either "sõda" or "kaitsevägi".'

In [35]:
df[df.text_splitted.str.contains('sõda')]

,text_splitted
471,lõpuks on 34 inimest nüüd teenistusest vabastatud. näiteks major paas kõikides sõdades osalenud igati laitmatu teenistusega mees lihtsalt lasti lahti kui paneme need numbrid kõik kokku siis tegelikult kaotus personali seas tuleb juba üsna suur kaitseväel on olnud probleeme et kõiki ajateenijaid õpetada sest kaadrit pole piisavalt olnud kuidas kaitsevägi nendes tingimustes pärast seda kolme lainet suudab oma ülesandeid korralikult täita
969,et need muudatused peaks sisse viima veel enne esimest lugemist ehk veel enne tänast päeva loomulikult kuna need muudatused saab sisse viia enne esimest lugemist ainult eelnõu algataja siis andis justiitsministeerium lahkelt selleks ka oma nõusoleku mis tõendab veel kord et mingeid erimeelsusi ega mingit sõda ei õiguskomisjoni istungitel ega töögrupis ei olnud konstruktiivse dialoogiga saime kõik küsimused lahendatud tõepoolest õiguskomisjon arutaski muudetud allikakaitse seaduse eelnõu uuesti juba 5. aprilli istungil see tähendab et allikakaitse seaduse eelnõu muutus ja toodi


In [36]:
df_resp

,text,id,score
0,vaktsineerimata siis see on probleem eesti sõjalisele tegevusele kaitseväe tegevusele nii sõja ajal kui ka nende ülesannete täitmisele mis kaitseväel rahuajal on,b2fcdba3-b747-4491-886f-ac9cf676b06a,0.904191
1,turvatunde ja selle et mõrtsukad ja pedofiilid ei jaluta rahulikult ringi,6b704253-4ee6-4882-b212-d77ef482c52b,0.898059
2,selleks et toetada neid inimesi seal ja hoida ära rändesurvet euroopale sealhulgas eestile,8232cef1-7d62-4b11-b833-ac857867f5b8,0.894815
3,järele siis valitsus võimaldab neile seda või eraldab selleks raha on teil olemas selleks mingi reserv,491f2fea-715a-40df-9a37-5caf5da193ca,0.891470
4,kohtadel kui saaks teie nägemuse sellest,20f58d68-c945-4a99-bd86-71f7ca7addec,0.891346
5,totaalne paradigma muutus või mis see siis nüüd on,73ca9d5b-485a-4113-b381-90ca841f7c5c,0.889357
6,õiguste kaitsega siin on riigikogu kaalumise koht,d14759b9-db94-4e30-9e08-6ee38c668e03,0.888034
7,paremad pensionid nii praegu kui ka tulevikus,850a771b-29d2-42ea-9c5c-4736e7a83211,0.887905
8,kas eelnõu on kavas selliste sätetega täiendada,0f0b0f0f-42cd-4034-a9a7-8723be852668,0.887397
9,vähegi võimalik nii et ma loodan et me kompenseerimise juurde üldse ei peagi tulema,62da1342-1322-42ac-b7af-0aa0eb84a600,0.886769


In [37]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis mainitakse riiki Ukraina?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is riik Ukraina


'Andmebaasi kohaselt mainitakse Ukrainat 20 tekstis.'

In [38]:
df[df.text_splitted.str.contains('ukraina')]

,text_splitted
191,arvatud rikastes põhjamaades see pärsib meie konkurentsivõimet ja paneb surve alla palgad kui kõik muud sisendhinnad on naabrite omadest kõrgemad siis ainus koht kust kokku hoida on töötajad just sellepärast nõuavad mõned tööandjad meile odavat tööjõudu ukrainast elektri ja gaasi hinda langetades anname ettevõtjatele võimaluse tõsta palkasid kaotamata konkurentsis välisturgudel eesmärk peaks olema muuta eesti kõige odavama energiaga riigiks meie piirkonnas milliseid maksupoliitilisi muudatusi me veel peame vajalikuks tee- ja veokimaks ei suurenda oluliselt riigieelarvet küll aga vähendab meie transpordiettevõtete
267,kolm maksuastet 16% 24% ja 33% lühikest aega ka 50% kui tuli võimule mart laar siis ta kehtestas ühtse 27%-lise maksu mis tähendas seda et rikkamad hakkasid vähem maksma ja väiksema sissetulekuga inimesed tunduvalt rohkem ma ei saa siiamaani aru miks me ei taha minna sama teed nagu muu euroopa miks me tahame seista kogu aeg ühel pulgal venemaaga kus on nagu meil proportsionaalne ja ühetaoline tulumaks täpselt samuti on ka bulgaarias rumeenias ukrainas ja ma ei tea bahama saartel


In [39]:
df_resp

,text,id,score
0,palun protseduuriline küsimus igor gräzin,1287b45c-e908-4a86-ad69-865f58cdc1c4,0.888185
1,vähegi võimalik nii et ma loodan et me kompenseerimise juurde üldse ei peagi tulema,62da1342-1322-42ac-b7af-0aa0eb84a600,0.882270
2,ei ole selleks mingit kohustust aga minu küsimus miks seda kohustust selle eelnõuga neile uuesti ei pandud,62276b9c-f07d-48c0-8a3b-a244b855b90f,0.881950
3,annavad oma parima selle nimel et tuleks väga hea seadus,bb135145-fd24-498f-b1c3-332dd81d1c20,0.881867
4,austatud minister palun vabandust kas te soovite lisaaega,b10dc9e0-311f-4c99-87e4-a1a991628c98,0.880985
5,siin eelnõus ei ole palun seda eelnõu võtta sellisena nagu see on esitatud,9fefa413-22a5-4d26-9ead-f8fc6bd482b4,0.879858
6,kohtadel kui saaks teie nägemuse sellest,20f58d68-c945-4a99-bd86-71f7ca7addec,0.878597
7,võib-olla saame väga hea seaduse,1cc7e916-c586-4c61-bac9-60050e10857d,0.878050
8,täpsustav küsimus andres metsoja palun,329f9eda-e652-4a42-9d35-2a9bf1037574,0.877774
9,järele siis valitsus võimaldab neile seda või eraldab selleks raha on teil olemas selleks mingi reserv,491f2fea-715a-40df-9a37-5caf5da193ca,0.877581


In [40]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis räägitakse kunstist või kunstiteosest või kunstilisest objektist?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is kunst OR kunstiteos OR kunstiline objekt


'Kunstist, kunstiteosest või kunstilisest objektist räägitakse andmebaasis kokku 20 tekstis.'

In [41]:
df[df.text_splitted.str.contains('kunst')]

,text_splitted
929,aitäh lugupeetud juhataja lugupeetud minister ka minu küsimus puudutab võlaõigusseaduse muudatusi mis on seotud kahju hüvitamisega eesti kohtupraktikas on üks selline juhtum et kadri k filmi m. tõlkes great ehk 'suurepärane' puhul ei olnud kohtul muud võimalust kui see film ära keelata kuigi tegelikult oleks lahenduseks olnud ka kahju hüvitamine kas selle eelnõu puhul oleks võimalik öelda et edaspidi sellistel juhtumitel mingi kultuuri- või ka kunstilise objekti filmi ükskõik mis asja ärakeelamise asemel saaks kasutada kahju hüvitamist kui ei siis


In [42]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis räägitakse arvutitest või IT-st?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is arvutitest OR IT


'20 tekstis räägitakse arvutitest või IT-st.'

In [43]:
df[df.text_splitted.str.contains('arvuti')]

,text_splitted
13,peale 3 miljardit tundi mis tegi ühe maksumaksja kohta keskmiseks ajakuluks 27 tundi aastas eesti on aga maailmas läbi löönud tugeva it-riigina ja siin esitatakse üle 98% tuludeklaratsioonidest interneti teel täiendav maksuaste ei ole arvutiprogrammi jaoks mingisugune takistus eesti on olnud aastaid sissetulekute jaotuse poolest pigem euroopa liidu ebavõrdsemate riikide hulgas seda näitab gini indeks mis sobib kõige paremini iseloomustama ühiskondade sotsiaalset ebavõrdsust ja kihistumist mida madalam on gini indeksi väärtus seda võrdsem on ühiskonna majanduslik ja sotsiaalne seisund arenenud
75,ei ole varsti enam optimaalne lahendus ja riigid peaksid mõtlema muule lahendusele see eristamine on kulukas see on võib-olla ebavajalik ja see ei pruugi mingist hetkest alates ka õiglustundega vastavuses olla peame arvestama ka sellega et diginomaadluski on kasvamas diginomaadid on sellised inimesed kuidas öelda mees ja arvuti tüüpi oü-de asutajad kes reisivad mööda maailma ja kelle töötegemise asukoht ei ole üldse oluline nemad teevad tööd internetis see tähendab seda et riikide sotsiaalkindlustussüsteemid on löögi all neli aastat tagasi hinnati


In [44]:
df_resp

,text,id,score
0,kohtadel kui saaks teie nägemuse sellest,20f58d68-c945-4a99-bd86-71f7ca7addec,0.904597
1,nad ei oota seda jah,6ac8b087-bb92-45ec-ae2f-4bae5b850ff2,0.898503
2,aitäh härra kõva kui vana te olete,179e0c34-8abb-48ec-a4b5-2b8a4a05e36c,0.895899
3,konkreetset lahendit internetiväljaannete puhul 14. see on ka kättesaadav,b9e184b4-a86c-42dc-a7a6-529746c79506,0.893209
4,keegi kontrollib ka selliste juhtumite puhul kohtuotsuste vastavust toimepandud tegudele,b3382203-980f-497a-8186-cf70fc40c8c4,0.893205
5,et 13 miljonit digiteenuste maksuraha kätte saada kaks küsimust ühes,0fc3189a-c2b9-4dbf-a2ec-73ef46711098,0.893149
6,võib-olla seda oleks pidanud küsima päevakorra kinnitamise juures aga ma vaatasin et,be3dc4c4-b322-446a-a015-b2ae523e5cb2,0.893003
7,palun protseduuriline küsimus igor gräzin,1287b45c-e908-4a86-ad69-865f58cdc1c4,0.892655
8,siin eelnõus ei ole palun seda eelnõu võtta sellisena nagu see on esitatud,9fefa413-22a5-4d26-9ead-f8fc6bd482b4,0.891959
9,totaalne paradigma muutus või mis see siis nüüd on,73ca9d5b-485a-4113-b381-90ca841f7c5c,0.891195


In [45]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis räägitakse infosüsteemide arendamisest?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is infosüsteemide arendamine


'Arhiivis on 20 teksti, kus räägitakse infosüsteemide arendamisest.'

In [46]:
df_resp

,text,id,score
0,saadavad nii et seadus on mõeldud eelkõige informatsiooni jagamiseks aga kindlasti ka usaldusväärsuse suurendamiseks,cab6974c-1c78-4170-befd-3a8f86e02bc9,0.894582
1,on plaanis kogu seda andmebaasi niimoodi ühtlustada et kehtestataks üks aruannete kuupäev,99acf739-1550-487e-a514-a91ba5db2c95,0.891507
2,küll aga on võimalik muuta riigiteenuseid läbipaistvamaks ja automaatsemaks kui me mõistlikult arendaksime oma e-riiki ja kasutaksime rohkem tehisintellekti aitäh,c09ece17-240e-4513-ae0a-b5e9e10689c4,0.890537
3,aitäh lõpetan selle küsimuse käsitlemise meil rohkem ei ole registreeritud teemasid,85a6b2a9-c2e5-47af-96f3-b7ea58cb7810,0.890372
4,et 13 miljonit digiteenuste maksuraha kätte saada kaks küsimust ühes,0fc3189a-c2b9-4dbf-a2ec-73ef46711098,0.887369
5,aitäh lõpetan selle küsimuse käsitlemise,739f9ba9-be84-4a3a-ba4f-65a3998c9112,0.885920
6,aitäh lõpetan selle küsimuse käsitlemise,44dc034a-5a14-4c00-8044-1135cb06327b,0.885920
7,aitäh lõpetan selle küsimuse käsitlemise,1bd47684-4534-48c3-947a-a7a1439170cd,0.885920
8,aitäh lõpetan selle küsimuse käsitlemise,88edc9a9-0777-40ca-9b35-d2c7325f84a9,0.885920
9,aitäh lõpetan selle küsimuse käsitlemise,68e35590-b61e-48a3-b279-eba87a5c4185,0.885920
